### All important libraries are imported below

In [62]:
import pandas as pd 
import numpy as np
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import cross_val_score,GridSearchCV
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,recall_score,f1_score

In [63]:
# reads test and train data
df_train=pd.read_csv('data/train.csv')
df_test=pd.read_csv('data/test.csv')

In [64]:
# overview the data what columns they have 
df_train

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [65]:
# counts of each null column 
df_train.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

# shows the distribution of values of each column

In [66]:
for column in df_train.columns:
    display(df_train[column].value_counts())
    display(df_train[column].value_counts())
    print()

0001_01    1
6136_01    1
6141_01    1
6139_06    1
6139_05    1
          ..
3126_01    1
3124_03    1
3124_02    1
3124_01    1
9280_02    1
Name: PassengerId, Length: 8693, dtype: int64

0001_01    1
6136_01    1
6141_01    1
6139_06    1
6139_05    1
          ..
3126_01    1
3124_03    1
3124_02    1
3124_01    1
9280_02    1
Name: PassengerId, Length: 8693, dtype: int64

Earth     4602
Europa    2131
Mars      1759
Name: HomePlanet, dtype: int64

Earth     4602
Europa    2131
Mars      1759
Name: HomePlanet, dtype: int64

False    5439
True     3037
Name: CryoSleep, dtype: int64

...

0.0       5495
1.0        139
2.0         70
3.0         56
5.0         51
          ... 
408.0        1
876.0        1
2891.0       1
2102.0       1
3235.0       1
Name: VRDeck, Length: 1306, dtype: int64

Gollux Reedall        2
Elaney Webstephrey    2
Grake Porki           2
Sus Coolez            2
Apix Wala             2
                     ..
Jamela Griffy         1
Hardy Griffy          1
Salley Mckinn         1
Mall Frasp            1
Propsh Hontichre      1
Name: Name, Length: 8473, dtype: int64

Gollux Reedall        2
Elaney Webstephrey    2
Grake Porki           2
Sus Coolez            2
Apix Wala             2
                     ..
Jamela Griffy         1
Hardy Griffy          1
Salley Mckinn         1
Mall Frasp            1
Propsh Hontichre      1
Name: Name, Length: 8473, dtype: int64

True     4378
False    4315
Name: Transported, dtype: int64

True     4378
False    4315
Name: Transported, dtype: int64

### as we see result above , some of the columns distributed badly, therefore if we take a look at these columns carefully, most of them are numeric and the most used variable is 0 ,  so we can fill these nulls with 0.

In [67]:
# for this column most used variable is "False"
df_train['VIP']=df_train['VIP'].fillna(False)
df_test['VIP']=df_test['VIP'].fillna(False)

In [68]:
df_train['RoomService']=df_train['RoomService'].fillna(0.0)
df_test['RoomService']=df_test['RoomService'].fillna(0.0)

In [69]:
df_train['FoodCourt']=df_train['FoodCourt'].fillna(0.0)
df_test['FoodCourt']=df_test['FoodCourt'].fillna(0.0)

In [70]:
df_train['ShoppingMall']=df_train['ShoppingMall'].fillna(0.0)
df_test['ShoppingMall']=df_test['ShoppingMall'].fillna(0.0)

In [71]:
df_train['Spa']=df_train['Spa'].fillna(0.0)
df_test['Spa']=df_test['Spa'].fillna(0.0)

In [72]:
df_train['VRDeck']=df_train['VRDeck'].fillna(0.0)
df_test['VRDeck']=df_test['VRDeck'].fillna(0.0)

### Rate of remainging data after all nulls are dropped 

In [73]:
len(df_train.dropna())/len(df_train)

0.8715058092718279

In [74]:
len(df_test.dropna())/len(df_test)

0.8765489829319617

In [75]:
df_train.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Name            200
Transported       0
dtype: int64

In [82]:
df_train['HomePlanet']=df_train['HomePlanet'].fillna('Earth')
df_train['CryoSleep']=df_train['CryoSleep'].fillna(False)
df_train['Destination']=df_train['Destination'].fillna('TRAPPIST-1e')
df_train['Age']=df_train['Age'].fillna(24.0)

In [83]:
# drop all remaing nulls from train 
df_train=df_train.dropna()

1.0

In [84]:
df_test['HomePlanet']=df_test['HomePlanet'].fillna('Earth')
df_test['CryoSleep']=df_test['CryoSleep'].fillna(False)
df_test['Cabin']=df_test['Cabin'].fillna('G/160/P')
df_test['Destination']=df_test['Destination'].fillna('TRAPPIST-1e')
df_test['Age']=df_test['Age'].fillna(25.8)   # mean 28.5 # top20 mean 25.8

In [85]:
#df_test.RoomService.value_counts()

In [86]:
#df_train.Age.value_counts().to_frame().head(20)

In [87]:
df_train['Cabin_1']=0
df_train['Cabin_2']=0
df_train['Cabin_3']=0
for index,row in df_train.iterrows():
    tmp_cabin_list=row['Cabin'].split('/')
    df_train.at[index,'Cabin_1']=tmp_cabin_list[0]
    df_train.at[index,'Cabin_2']=tmp_cabin_list[1]
    df_train.at[index,'Cabin_3']=tmp_cabin_list[2]

/var/folders/x2/dxy3xlpn5jvcpyfkpvx2zm2r0000gn/T/ipykernel_1959/2886987449.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['Cabin_1']=0
/var/folders/x2/dxy3xlpn5jvcpyfkpvx2zm2r0000gn/T/ipykernel_1959/2886987449.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['Cabin_2']=0
/var/folders/x2/dxy3xlpn5jvcpyfkpvx2zm2r0000gn/T/ipykernel_1959/2886987449.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [88]:
df_test['Cabin_1']=0
df_test['Cabin_2']=0
df_test['Cabin_3']=0
for index,row in df_test.iterrows():
    tmp_cabin_list=row['Cabin'].split('/')
    df_test.at[index,'Cabin_1']=tmp_cabin_list[0]
    df_test.at[index,'Cabin_2']=tmp_cabin_list[1]
    df_test.at[index,'Cabin_3']=tmp_cabin_list[2]

In [89]:
df_train.drop(columns=['Cabin'],inplace=True)
df_test.drop(columns=['Cabin'],inplace=True)

/var/folders/x2/dxy3xlpn5jvcpyfkpvx2zm2r0000gn/T/ipykernel_1959/2654294975.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.drop(columns=['Cabin'],inplace=True)


In [90]:
df_train['Cabin_2']=df_train.Cabin_2.apply(pd.to_numeric)
df_test['Cabin_2']=df_test.Cabin_2.apply(pd.to_numeric)

/var/folders/x2/dxy3xlpn5jvcpyfkpvx2zm2r0000gn/T/ipykernel_1959/341266545.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['Cabin_2']=df_train.Cabin_2.apply(pd.to_numeric)


# Work with Name columns

In [91]:
all_name=list(set(df_train.Name).union(set(df_test.Name)))

In [92]:
names_with_id=dict()
for i in range(len(all_name)):
    names_with_id[all_name[i]]=i

In [93]:
df_train['Name_encoded']=0
for index,row in df_train.iterrows():
    df_train.at[index,'Name_encoded']=names_with_id[row['Name']]

/var/folders/x2/dxy3xlpn5jvcpyfkpvx2zm2r0000gn/T/ipykernel_1959/3089315358.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['Name_encoded']=0


In [94]:
df_test['Name_encoded']=0
for index,row in df_test.iterrows():
    df_test.at[index,'Name_encoded']=names_with_id[row['Name']]

In [95]:
df_train.drop(columns=['Name'],inplace=True)
df_test.drop(columns=['Name'],inplace=True)

/var/folders/x2/dxy3xlpn5jvcpyfkpvx2zm2r0000gn/T/ipykernel_1959/1324597439.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.drop(columns=['Name'],inplace=True)


In [96]:
#df_train.drop(columns=['Name_encoded'],inplace=True)
#df_test.drop(columns=['Name_encoded'],inplace=True)

In [97]:
# Multiple categorical columns
categorical_cols = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP','Cabin_1','Cabin_3']

df_train=pd.get_dummies(df_train, columns=categorical_cols).copy()
df_test=pd.get_dummies(df_test, columns=categorical_cols).copy()

In [98]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

### For local test train the train data 

In [99]:
# splitting X and y into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_train.drop(columns=['PassengerId','Transported']),
                                                    df_train['Transported'], test_size=0.3, random_state=42)


In [100]:
# training the model on training set
gnb = GaussianNB()
gnb.fit(X_train, y_train)

# making predictions on the testing set
y_pred = gnb.predict(X_test)
  
# comparing actual response values (y_test) with predicted response values (y_pred)
print("Gaussian Naive Bayes model accuracy(in %):", metrics.accuracy_score(y_test, y_pred)*100)

Gaussian Naive Bayes model accuracy(in %): 75.29128163921254


# XgBoost Algorithm and HyperTuning

### hyperparameter tuning

In [101]:
# import packages for hyperparameters tuning
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [102]:
space={'max_depth': hp.quniform("max_depth", 3, 10, 1),
        'gamma': hp.uniform ('gamma', 1,20),
        'learning_rate': hp.quniform('learning_rate', 0,2,0.01),
        'max_delta_step':hp.uniform ('max_delta_step', 0,10),
        'reg_alpha' : hp.quniform('reg_alpha', 0,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'min_child_weight' : hp.uniform('min_child_weight', 0, 10),
        'n_estimators': 100,
        'seed': 10
    }    

In [103]:
def objective(space):
    clf=xgb.XGBClassifier(
                    n_estimators =space['n_estimators'],
                    max_depth = int(space['max_depth']), 
                    gamma = space['gamma'],
                    reg_alpha = space['reg_alpha'],
                    min_child_weight=int(space['min_child_weight']),
                    max_delta_step=int(space['max_delta_step']),
                    learning_rate=space['learning_rate'],
                    seed=space['seed'],
                    eval_metric='auc',
                    early_stopping_rounds=15
    )
    
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    clf.fit(X_train, y_train,
            eval_set=evaluation,
            verbose=False)
    

    pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, pred>0.5)
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK }

In [104]:
trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 300,
                        trials = trials)

SCORE:                                                                          
0.7697870630775412                                                              
SCORE:                                                                          
0.7766171153073523                                                              
SCORE:                                                                          
0.7850542386500603                                                              
SCORE:                                                                          
0.7657693852952994                                                              
SCORE:                                                                          
0.7621534752912816                                                              
SCORE:                                                                          
0.773804740859783                                                               
SCORE:                      

0.791080755323423                                                               
SCORE:                                                                          
0.7577340297308156                                                              
SCORE:                                                                          
0.7914825231016472                                                              
SCORE:                                                                          
0.7794294897549217                                                              
SCORE:                                                                          
0.7661711530735235                                                              
SCORE:                                                                          
0.7938931297709924                                                              
SCORE:                                                                          
0.791080755323423           

SCORE:                                                                          
0.7874648453194054                                                              
SCORE:                                                                          
0.782241864202491                                                               
SCORE:                                                                          
0.7878666130976296                                                              
SCORE:                                                                          
0.7959019686621133                                                              
SCORE:                                                                          
0.7826436319807152                                                              
SCORE:                                                                          
0.8011249497790277                                                              
SCORE:                      

0.8055443953394937                                                              
SCORE:                                                                          
0.8035355564483728                                                              
SCORE:                                                                          
0.8023302531137003                                                              
SCORE:                                                                          
0.8083567697870631                                                              
SCORE:                                                                          
0.7938931297709924                                                              
SCORE:                                                                          
0.8071514664523906                                                              
SCORE:                                                                          
0.7979108075532342          

SCORE:                                                                          
0.7946966653274408                                                              
SCORE:                                                                          
0.4961832061068702                                                              
SCORE:                                                                          
0.8011249497790277                                                              
SCORE:                                                                          
0.8043390920048212                                                              
SCORE:                                                                          
0.7979108075532342                                                              
SCORE:                                                                          
0.7955002008838891                                                              
SCORE:                      

0.7858577742065086                                                              
SCORE:                                                                          
0.7946966653274408                                                              
SCORE:                                                                          
0.7959019686621133                                                              
SCORE:                                                                          
0.8003214142225793                                                              
SCORE:                                                                          
0.7505022097227803                                                              
SCORE:                                                                          
0.8031337886701486                                                              
SCORE:                                                                          
0.8051426275612696          

In [108]:
print("The best hyperparameters are : ","\n")
print(best_hyperparams)

The best hyperparameters are :  

{'gamma': 3.960627464795948, 'learning_rate': 0.16, 'max_delta_step': 4.639178550275831, 'max_depth': 8.0, 'min_child_weight': 1.1863205634697773, 'reg_alpha': 3.0, 'reg_lambda': 0.5048858425550264}


In [109]:
clf=xgb.XGBClassifier(
    max_depth = int(best_hyperparams['max_depth']),
    gamma = best_hyperparams['gamma'],
    reg_alpha = best_hyperparams['reg_alpha'],
    min_child_weight=best_hyperparams['min_child_weight'],
    learning_rate=best_hyperparams['learning_rate'],
    max_delta_step=best_hyperparams['max_delta_step'],
    
    
)
evaluation = [( X_train, y_train), ( X_test, y_test)]
    
clf.fit(X_train, y_train,
        eval_set=evaluation,
        verbose=False)
    

y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred>0.5)
print ("SCORE:", accuracy)

SCORE: 0.8079550020088389


In [110]:
print(confusion_matrix(y_test, y_pred), ": is the confusion matrix")
print(accuracy_score(y_test, y_pred), ": is the accuracy score")
print(precision_score(y_test, y_pred), ": is the precision score")
print(recall_score(y_test, y_pred), ": is the recall score")
print(f1_score(y_test, y_pred), ": is the f1 score")

[[ 981  254]
 [ 224 1030]] : is the confusion matrix
0.8079550020088389 : is the accuracy score
0.8021806853582555 : is the precision score
0.8213716108452951 : is the recall score
0.8116627265563435 : is the f1 score


# For submittion

In [467]:
y_pred_sub = clf.predict(df_test.drop(columns=['PassengerId']))

In [468]:
y_pred_sub

array([1, 0, 1, ..., 1, 1, 1])

In [469]:
y = 0 < y_pred_sub
y

array([ True, False,  True, ...,  True,  True,  True])

# Save result

In [448]:
df_submit=df_test[['PassengerId']].copy()
df_submit['Transported']=y

In [470]:
df_submit.to_csv('data/submit_3.csv',index = False)

In [471]:
df_submit

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True
